# 🧠 Comparing LLM-as-judge Functions

Our goal is to compare different prompts and settings for LLM-as-judge.

In [ ]:
# Auto-reload for dev
%load_ext autoreload
%autoreload 2

import os
import json
from pathlib import Path
from text2sql_eval_toolkit.utils import (
    get_benchmark_info,
    get_default_eval_filename,
    get_question,
)
from text2sql_eval_toolkit.analysis.report_tools import collect_results
from text2sql_eval_toolkit.evaluation.llm_as_judge import (
    load_llm_judge_config,
    evaluate_sql_prediction_with_llm,
)

from dotenv import load_dotenv

# Load environment variables for LLM API credentials from project root
# The .env file should be in the project root directory (works when cwd is project root or notebooks/)
project_root = Path.cwd() if (Path.cwd() / ".env").exists() else Path.cwd().parent
env_path = project_root / '.env'
load_dotenv(env_path, override=True)

print(f"File exists: {env_path.exists()}")
print(f"Loading .env from: {env_path}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [ ]:
import json
import numpy as np
from collections import defaultdict
from pathlib import Path

# For Jupyter notebook display
try:
    from IPython.display import Markdown, display

    JUPYTER_AVAILABLE = True
except ImportError:
    JUPYTER_AVAILABLE = False


def create_confusion_matrix(pred_eval_data, key1, key2, pipeline_id=None):
    """
    Create a confusion matrix between two evaluation metrics.

    Args:
        pred_eval_data: Array of objects with predictions
        key1: First evaluation metric key (e.g., "llm_score")
        key2: Second evaluation metric key (e.g., "llm_judge_no_gt_v1_score")
        pipeline_id: Specific pipeline ID to analyze (if None, uses ALL pipeline IDs)

    Returns:
        confusion_matrix: 2x2 numpy array
        agreement_percentage: Percentage of agreement between the two metrics
    """

    # Collect pairs of scores
    score_pairs = []

    for item in pred_eval_data:
        if "predictions" in item:
            predictions = item["predictions"]

            # If no specific pipeline_id provided, use ALL available pipeline IDs
            if pipeline_id is None:
                pipeline_ids = list(predictions.keys())
            else:
                pipeline_ids = [pipeline_id] if pipeline_id in predictions else []

            for pid in pipeline_ids:
                if pid in predictions:
                    evaluation = predictions[pid].get("evaluation", {})

                    if key1 in evaluation and key2 in evaluation:
                        score1 = evaluation[key1]
                        score2 = evaluation[key2]

                        # Convert to binary (assuming 0.0 and 1.0 are the only values)
                        binary1 = int(score1 > 0.5)  # Convert to 0 or 1
                        binary2 = int(score2 > 0.5)  # Convert to 0 or 1

                        score_pairs.append((binary1, binary2))

    if not score_pairs:
        print(f"No data found for keys '{key1}' and '{key2}'")
        return None, None

    # Create confusion matrix
    # Format: [[TN, FP], [FN, TP]]
    # Where True/False refers to key1, Positive/Negative refers to key2
    confusion_matrix = np.zeros((2, 2), dtype=int)

    for score1, score2 in score_pairs:
        confusion_matrix[score1][score2] += 1

    # Calculate agreement percentage
    total_samples = len(score_pairs)
    agreements = sum(1 for s1, s2 in score_pairs if s1 == s2)
    agreement_percentage = (
        (agreements / total_samples) * 100 if total_samples > 0 else 0
    )

    return confusion_matrix, agreement_percentage


def print_confusion_matrix_analysis(pred_eval_data, key1, key2, pipeline_id=None):
    """
    Print a detailed analysis of the confusion matrix between two evaluation metrics.
    By default, analyzes ALL pipeline IDs combined.
    """

    print(f"\n{'=' * 60}")
    print(f"CONFUSION MATRIX ANALYSIS")
    print(f"Key 1 (rows): {key1}")
    print(f"Key 2 (cols): {key2}")

    if pipeline_id:
        print(f"Pipeline ID: {pipeline_id}")
    else:
        # Show which pipeline IDs are being analyzed
        pipeline_ids = set()
        for item in pred_eval_data:
            if "predictions" in item:
                pipeline_ids.update(item["predictions"].keys())
        pipeline_ids = sorted(list(pipeline_ids))
        print(f"Pipeline IDs: ALL ({len(pipeline_ids)} total: {pipeline_ids})")

        # Show per-pipeline agreement summary
        if len(pipeline_ids) > 1:
            pipeline_stats = get_pipeline_agreement_summary(pred_eval_data, key1, key2)
            if pipeline_stats:
                print(f"\nPer-Pipeline Agreement Summary:")
                print(
                    f"{'Pipeline ID':<20} {'Samples':<8} {'Agreements':<12} {'Agreement %':<12}"
                )
                print(f"{'-' * 20} {'-' * 8} {'-' * 12} {'-' * 12}")
                for stats in pipeline_stats:
                    print(
                        f"{stats['pipeline_id']:<20} {stats['total_samples']:<8} {stats['agreements']:<12} {stats['agreement_percentage']:<12.1f}%"
                    )

    print(f"{'=' * 60}")

    confusion_matrix, agreement_percentage = create_confusion_matrix(
        pred_eval_data, key1, key2, pipeline_id
    )

    if confusion_matrix is None:
        return

    # Print confusion matrix
    matrix_title = (
        "Combined Confusion Matrix:" if pipeline_id is None else "Confusion Matrix:"
    )
    print(f"\n{matrix_title}")
    print(f"                 {key2}")
    print(f"                 0.0    1.0")
    print(f"{key1:12} 0.0  {confusion_matrix[0][0]:4d}   {confusion_matrix[0][1]:4d}")
    print(f"            1.0  {confusion_matrix[1][0]:4d}   {confusion_matrix[1][1]:4d}")

    # Calculate and print detailed statistics
    total = np.sum(confusion_matrix)
    both_zero = confusion_matrix[0][0]  # Both scores are 0.0
    both_one = confusion_matrix[1][1]  # Both scores are 1.0
    key1_high_key2_low = confusion_matrix[1][0]  # Key1=1.0, Key2=0.0
    key1_low_key2_high = confusion_matrix[0][1]  # Key1=0.0, Key2=1.0

    breakdown_title = (
        "Combined Detailed Breakdown:" if pipeline_id is None else "Detailed Breakdown:"
    )
    print(f"\n{breakdown_title}")
    print(f"Total samples: {total}")
    print(
        f"Both {key1}=0.0 and {key2}=0.0: {both_zero:4d} ({both_zero / total * 100:.1f}%)"
    )
    print(
        f"Both {key1}=1.0 and {key2}=1.0: {both_one:4d} ({both_one / total * 100:.1f}%)"
    )
    print(
        f"{key1}=1.0, {key2}=0.0: {key1_high_key2_low:4d} ({key1_high_key2_low / total * 100:.1f}%)"
    )
    print(
        f"{key1}=0.0, {key2}=1.0: {key1_low_key2_high:4d} ({key1_low_key2_high / total * 100:.1f}%)"
    )

    agreement_title = (
        "Combined Agreement Analysis:" if pipeline_id is None else "Agreement Analysis:"
    )
    print(f"\n{agreement_title}")
    print(f"Agreement percentage: {agreement_percentage:.2f}%")
    print(f"Disagreement percentage: {100 - agreement_percentage:.2f}%")

    return confusion_matrix, agreement_percentage


def get_pipeline_agreement_summary(pred_eval_data, key1, key2):
    """
    Get agreement statistics for each individual pipeline.

    Returns:
        List of dictionaries with pipeline stats
    """
    # Get all available pipeline IDs
    pipeline_ids = set()
    for item in pred_eval_data:
        if "predictions" in item:
            pipeline_ids.update(item["predictions"].keys())

    pipeline_ids = sorted(list(pipeline_ids))
    pipeline_stats = []

    for pipeline_id in pipeline_ids:
        confusion_matrix, agreement_percentage = create_confusion_matrix(
            pred_eval_data, key1, key2, pipeline_id
        )

        if confusion_matrix is not None:
            total_samples = np.sum(confusion_matrix)
            agreements = (
                confusion_matrix[0][0] + confusion_matrix[1][1]
            )  # Both 0.0 + Both 1.0

            pipeline_stats.append(
                {
                    "pipeline_id": pipeline_id,
                    "total_samples": total_samples,
                    "agreements": agreements,
                    "agreement_percentage": agreement_percentage,
                }
            )

    return pipeline_stats


def print_confusion_matrix_markdown(
    pred_eval_data, key1, key2, pipeline_id=None, display_in_notebook=True
):
    """
    Print a confusion matrix analysis in Markdown format for Jupyter notebooks.
    By default, analyzes ALL pipeline IDs combined.

    Args:
        display_in_notebook: If True and in Jupyter, renders the markdown nicely.
                           If False, just prints the raw markdown text.
    """

    confusion_matrix, agreement_percentage = create_confusion_matrix(
        pred_eval_data, key1, key2, pipeline_id
    )

    if confusion_matrix is None:
        message = "No data found for the specified keys."
        if display_in_notebook and JUPYTER_AVAILABLE:
            display(Markdown(f"**Error:** {message}"))
        else:
            print(message)
        return

    # Get pipeline info and per-pipeline stats
    if pipeline_id:
        pipeline_info = f"**Pipeline ID:** `{pipeline_id}`"
        pipeline_summary_table = ""
    else:
        pipeline_ids = set()
        for item in pred_eval_data:
            if "predictions" in item:
                pipeline_ids.update(item["predictions"].keys())
        pipeline_ids = sorted(list(pipeline_ids))
        pipeline_info = f"**Pipeline IDs:** ALL ({len(pipeline_ids)} total: {', '.join([f'`{pid}`' for pid in pipeline_ids])})"

        # Get per-pipeline agreement stats
        pipeline_stats = get_pipeline_agreement_summary(pred_eval_data, key1, key2)

        if pipeline_stats:
            pipeline_summary_table = f"""
### 📋 Per-Pipeline Agreement Summary

| Pipeline ID | Total Samples | Agreements | Agreement % |
|-------------|:-------------:|:----------:|:-----------:|
"""
            for stats in pipeline_stats:
                pipeline_summary_table += f"| `{stats['pipeline_id']}` | {stats['total_samples']:,} | {stats['agreements']:,} | {stats['agreement_percentage']:.1f}% |\n"

            pipeline_summary_table += "\n"
        else:
            pipeline_summary_table = ""

    # Calculate statistics
    total = np.sum(confusion_matrix)
    both_zero = confusion_matrix[0][0]
    both_one = confusion_matrix[1][1]
    key1_high_key2_low = confusion_matrix[1][0]
    key1_low_key2_high = confusion_matrix[0][1]

    # Generate Markdown
    markdown = f"""## 🔍 Confusion Matrix Analysis

**Metrics Compared:**
- **Key 1 (rows):** `{key1}`
- **Key 2 (cols):** `{key2}`

{pipeline_info}
{pipeline_summary_table}
### Confusion Matrix (Combined)

|                | **{key2}=0.0** | **{key2}=1.0** |
|----------------|:--------------:|:--------------:|
| **{key1}=0.0** |      {both_zero:,}       |      {key1_low_key2_high:,}       |
| **{key1}=1.0** |   {key1_high_key2_low:,}        |      {both_one:,}       |

### 📊 Detailed Breakdown (Combined)

| Category | Count | Percentage |
|----------|------:|-----------:|
| **Total samples** | {total:,} | 100.0% |
| Both metrics = 0.0 | {both_zero:,} | {both_zero / total * 100:.1f}% |
| Both metrics = 1.0 | {both_one:,} | {both_one / total * 100:.1f}% |
| `{key1}`=1.0, `{key2}`=0.0 | {key1_high_key2_low:,} | {key1_high_key2_low / total * 100:.1f}% |
| `{key1}`=0.0, `{key2}`=1.0 | {key1_low_key2_high:,} | {key1_low_key2_high / total * 100:.1f}% |

### 🎯 Agreement Analysis (Combined)

- **Agreement:** {agreement_percentage:.2f}% ({both_zero + both_one:,} out of {total:,} samples)
- **Disagreement:** {100 - agreement_percentage:.2f}% ({key1_high_key2_low + key1_low_key2_high:,} out of {total:,} samples)"""

    # Display in notebook or print as text
    if display_in_notebook and JUPYTER_AVAILABLE:
        display(Markdown(markdown))
    else:
        print(markdown)

    return confusion_matrix, agreement_percentage


def analyze_all_pipeline_combinations(pred_eval_data, key1, key2):
    """
    Analyze confusion matrix for all available pipeline IDs.
    """

    # Get all available pipeline IDs
    pipeline_ids = set()
    for item in pred_eval_data:
        if "predictions" in item:
            pipeline_ids.update(item["predictions"].keys())

    pipeline_ids = sorted(list(pipeline_ids))

    if not pipeline_ids:
        print("No pipeline IDs found in the data")
        return

    print(f"\nFound {len(pipeline_ids)} pipeline ID(s): {pipeline_ids}")

    # Analyze each pipeline
    for pipeline_id in pipeline_ids:
        print_confusion_matrix_analysis(pred_eval_data, key1, key2, pipeline_id)


def analyze_all_pipeline_combinations_markdown(
    pred_eval_data, key1, key2, display_in_notebook=True
):
    """
    Analyze confusion matrix for all available pipeline IDs in Markdown format.

    Args:
        display_in_notebook: If True and in Jupyter, renders the markdown nicely.
                           If False, just prints the raw markdown text.
    """

    # Get all available pipeline IDs
    pipeline_ids = set()
    for item in pred_eval_data:
        if "predictions" in item:
            pipeline_ids.update(item["predictions"].keys())

    pipeline_ids = sorted(list(pipeline_ids))

    if not pipeline_ids:
        message = "No pipeline IDs found in the data"
        if display_in_notebook and JUPYTER_AVAILABLE:
            display(Markdown(f"**Error:** {message}"))
        else:
            print(message)
        return

    # Header markdown
    header_markdown = f"""# 📋 Pipeline-by-Pipeline Analysis

**Found {len(pipeline_ids)} pipeline ID(s):** {", ".join([f"`{pid}`" for pid in pipeline_ids])}

---"""

    if display_in_notebook and JUPYTER_AVAILABLE:
        display(Markdown(header_markdown))
    else:
        print(header_markdown)

    # Analyze each pipeline
    for i, pipeline_id in enumerate(pipeline_ids, 1):
        pipeline_header = f"## Pipeline {i}: `{pipeline_id}`"

        if display_in_notebook and JUPYTER_AVAILABLE:
            display(Markdown(pipeline_header))
        else:
            print(f"\n{pipeline_header}\n")

        # Call the individual analysis function
        print_confusion_matrix_markdown(
            pred_eval_data, key1, key2, pipeline_id, display_in_notebook
        )

        # Add separator between pipelines (except for the last one)
        if i < len(pipeline_ids):
            if display_in_notebook and JUPYTER_AVAILABLE:
                display(Markdown("---"))
            else:
                print("\n---\n")

In [68]:
benchmark_id = "bird_mini_dev_sqlite"
benchmark_info = get_benchmark_info(benchmark_id)
predictions_path = benchmark_info["predictions_path"]
eval_path = Path(get_default_eval_filename(predictions_path))
print(f"Evaluation file path: {eval_path}")
pred_eval_data = json.load(eval_path.open("r"))

Evaluation file path: /data/code/text2sql/text2sql-eval-toolkit/data/results/bird_mini_dev_sqlite-predictions_eval.json


In [69]:
#  "llm_judge_default_config_score", "llm_judge_alt_config_score", "llm_judge_no_gt_v1_score", "llm_judge_no_gt_v2_score"
key1 = "llm_score"  # "llm_judge_default_config_score" # "llm_score"
key2 = "llm_judge_no_gt_v1_score"  # "llm_judge_no_gt_v1_score" # "llm_judge_default_config_score"

confusion_matrix, agreement_percentage = print_confusion_matrix_markdown(
    pred_eval_data,
    key1,
    key2,
    # "wxai:ibm/granite-3-3-8b-instruct-greedy-zero-shot-chatapi",
)

## 🔍 Confusion Matrix Analysis

**Metrics Compared:**
- **Key 1 (rows):** `llm_score`
- **Key 2 (cols):** `llm_judge_no_gt_v1_score`

**Pipeline IDs:** ALL (7 total: `wxai:ibm/granite-3-3-8b-instruct-greedy-zero-shot`, `wxai:ibm/granite-3-3-8b-instruct-greedy-zero-shot-chatapi`, `wxai:meta-llama/llama-3-3-70b-instruct-greedy-zero-shot`, `wxai:meta-llama/llama-3-3-70b-instruct-greedy-zero-shot-chatapi`, `wxai:meta-llama/llama-4-maverick-17b-128e-instruct-fp8-greedy-zero-shot`, `wxai:meta-llama/llama-4-maverick-17b-128e-instruct-fp8-greedy-zero-shot-chatapi`, `wxai:openai/gpt-oss-120b-greedy-zero-shot-chatapi`)

### 📋 Per-Pipeline Agreement Summary

| Pipeline ID | Total Samples | Agreements | Agreement % |
|-------------|:-------------:|:----------:|:-----------:|
| `wxai:ibm/granite-3-3-8b-instruct-greedy-zero-shot` | 139 | 106 | 76.3% |
| `wxai:ibm/granite-3-3-8b-instruct-greedy-zero-shot-chatapi` | 134 | 105 | 78.4% |
| `wxai:meta-llama/llama-3-3-70b-instruct-greedy-zero-shot` | 141 | 109 | 77.3% |
| `wxai:meta-llama/llama-3-3-70b-instruct-greedy-zero-shot-chatapi` | 137 | 114 | 83.2% |
| `wxai:meta-llama/llama-4-maverick-17b-128e-instruct-fp8-greedy-zero-shot` | 138 | 118 | 85.5% |
| `wxai:meta-llama/llama-4-maverick-17b-128e-instruct-fp8-greedy-zero-shot-chatapi` | 132 | 111 | 84.1% |
| `wxai:openai/gpt-oss-120b-greedy-zero-shot-chatapi` | 128 | 108 | 84.4% |


### Confusion Matrix (Combined)

|                | **llm_judge_no_gt_v1_score=0.0** | **llm_judge_no_gt_v1_score=1.0** |
|----------------|:--------------:|:--------------:|
| **llm_score=0.0** |      201       |      121       |
| **llm_score=1.0** |   57        |      570       |

### 📊 Detailed Breakdown (Combined)

| Category | Count | Percentage |
|----------|------:|-----------:|
| **Total samples** | 949 | 100.0% |
| Both metrics = 0.0 | 201 | 21.2% |
| Both metrics = 1.0 | 570 | 60.1% |
| `llm_score`=1.0, `llm_judge_no_gt_v1_score`=0.0 | 57 | 6.0% |
| `llm_score`=0.0, `llm_judge_no_gt_v1_score`=1.0 | 121 | 12.8% |

### 🎯 Agreement Analysis (Combined)

- **Agreement:** 81.24% (771 out of 949 samples)
- **Disagreement:** 18.76% (178 out of 949 samples)